# COGS 118A - Project Checkpoint

# Names

Hopefully your team is at least this good. Obviously you should replace these with your names.

- Pelé
- Diego Maradonna
- Johan Cruyff
- Roberto Carlos
- Franz Beckenbaur

# Abstract 
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize: 
- what your goal/problem is
- what the data used represents and how they are measured
- what you will be doing with the data
- how performance/success will be measured

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 2 or 3 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data
- Link: https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre
- Number of observations: 50005 Number of variables: 15
- An observation consist of:
    - instance_id: A unique identifier for each instance.
    - popularity: The popularity of the track.
    - acousticness: A measure of the acoustic quality of the track.
    - danceability: A measure of the danceability of the track.
    - duration_ms: The duration of the track in milliseconds.
    - energy: A measure of the energy of the track.
    - instrumentalness: A measure of the instrumentalness of the track.
    - key: The key of the track.
    - liveness: A measure of the liveness of the track.
    - loudness: The loudness of the track.
    - mode: The mode of the track (Major or Minor).
    - speechiness: A measure of the speechiness of the track.
    - tempo: The tempo of the track.
    - valence: A measure of the musical positiveness conveyed by a track.
    - music_genre: The genre of the music.
- Critical variables: 
    - music_genre: This is likely the target variable because we are building a model to predict the genre of a track based on its other characteristics.

    - acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, valence: These are all numerical features that describe different aspects of the track, which could be important predictors in a model.

    - key, mode: These are categorical features that could also be important, depending on the specific genre we are trying to predict.
- Ways to represent:
    - Numerical variables (popularity, acousticness, danceability, duration_ms, energy, instrumentalness, liveness, loudness, speechiness, tempo, valence) are represented as floating-point numbers.

    - Categorical variables (key, mode, music_genre) are represented as strings.

    - The instance_id is a unique identifier for each observation, represented as a floating-point number. 


# Set Up

In [7]:
# import pandas & numpy library
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report


# Data Cleaning

In [8]:
path = "music_genre.csv"
music_data = pd.read_csv(path)

# Drop unrelated columns
music_data = music_data.drop(['artist_name', 'track_name', 'obtained_date'], axis=1)

# Replace missing values
music_data.replace(['?', -1], np.nan, inplace=True)

# Convert data types
music_data['duration_ms'] = music_data['duration_ms'].astype(float)
music_data['tempo'] = music_data['tempo'].astype(float)

# Standardize text data
music_data['key'] = music_data['key'].str.lower().str.strip()
music_data['mode'] = music_data['mode'].str.lower().str.strip()
music_data['music_genre'] = music_data['music_genre'].str.lower().str.strip()

# Handle NaN values (this is just one approach, you'll need to decide what's best for your use case)
music_data.dropna(inplace=True)

print(music_data.head())

   instance_id  popularity  acousticness  danceability  duration_ms  energy  \
1      46652.0        31.0       0.01270         0.622     218293.0   0.890   
2      30097.0        28.0       0.00306         0.620     215613.0   0.755   
3      62177.0        34.0       0.02540         0.774     166875.0   0.700   
4      24907.0        32.0       0.00465         0.638     222369.0   0.587   
6      43760.0        46.0       0.02890         0.572     214408.0   0.803   

   instrumentalness key  liveness  loudness   mode  speechiness    tempo  \
1          0.950000   d     0.124    -7.043  minor       0.0300  115.002   
2          0.011800  g#     0.534    -4.617  major       0.0345  127.994   
3          0.002530  c#     0.157    -4.498  major       0.2390  128.014   
4          0.909000  f#     0.157    -6.266  major       0.0413  145.036   
6          0.000008   b     0.106    -4.294  major       0.3510  149.995   

   valence music_genre  
1    0.531  electronic  
2    0.333  electr

# Proposed Solution

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Preliminary results

NEW SECTION!

Please show any preliminary results you have managed to obtain.

Examples would include:
- Analyzing the suitability of a dataset or alogrithm for prediction/solving your problem 
- Performing feature selection or hand-designing features from the raw data. Describe the features available/created and/or show the code for selection/creation
- Showing the performance of a base model/hyper-parameter setting.  Solve the task with one "default" algorithm and characterize the performance level of that base model.
- Learning curves or validation curves for a particular model
- Tables/graphs showing the performance of different models/hyper-parameters



# Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination.

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

# Team Expectations 

Put things here that cement how you will interact/communicate as a team, how you will handle conflict and difficulty, how you will handle making decisions and setting goals/schedule, how much work you expect from each other, how you will handle deadlines, etc...
* *Team Expectation 1*
* *Team Expectation 2*
* *Team Expecation 3*
* ...

# Project Timeline Proposal

UPDATE THE PROPOSAL TIMELINE ACCORDING TO WHAT HAS ACTUALLY HAPPENED AND HOW IT HAS EFFECTED YOUR FUTURE PLANS

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic (Pelé) | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets (Beckenbaur)  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data ,do some EDA (Maradonna) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin programming for project (Cruyff) | Discuss/edit project code; Complete project |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Carlos)| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem
